In [1]:
import numpy as np
import tensorflow as tf
from dataprovider import CIFAR10DataProvider
from tfutils.data import get_queue
from color_utils import preprocess

import matplotlib.pyplot as plt

% matplotlib inline

In [2]:
batch_size = 256
data_params = {
                # Cifar 10 data provider arguments
                'func': CIFAR10DataProvider,
                'data_path': '/datasets/cifar10/tfrecords',
                'group': 'train',
                'crop_size': 24,
                # TFRecords (super class) data provider arguments
                'file_pattern': 'train*.tfrecords',
                'batch_size': batch_size,
                'shuffle': False,
                'shuffle_seed': 6,
                'n_threads': 4,
            }

queue_params = {
                'queue_type': 'random',
                'batch_size': batch_size,
                'seed': 6,
                'capacity': batch_size * 10,
                'min_after_dequeue': batch_size * 5
            }

In [3]:
def get_data(func, queue_params=None, **data_params):
    data_provider = func(**data_params)
    input_ops = data_provider.init_ops()
    assert len(input_ops) == data_params['n_threads'], (len(input_ops), data_params['n_threads'])
    assert len(input_ops) > 0, len(input_ops)
    batch_size = data_params['batch_size']
    data_params['func'] = func
    enqueue_ops = []
    queue = get_queue(input_ops[0], shape_flag=batch_size!=1, **queue_params)
    for input_op in input_ops:
        # enqueue_ops.append(queue.enqueue_many(input_op))
        if batch_size == 1:
            enqueue_ops.append(queue.enqueue(input_op))
        else:
            enqueue_ops.append(queue.enqueue_many(input_op))
    tf.train.queue_runner.add_queue_runner(tf.train.queue_runner.QueueRunner(queue,
                                                                             enqueue_ops))
    if queue_params['batch_size'] == 1:
        inputs = queue.dequeue()
    else:
        inputs = queue.dequeue_many(queue_params['batch_size'])
    return data_params, [queue], inputs

In [4]:
data_params, [queue], inputs = get_data(queue_params=queue_params, **data_params)

INFO:tfutils:Using all metadata from saved source


In [6]:
data_l, gt_ab_313, prior_boost_nongray = tf.py_func(preprocess, [inputs['images']], [tf.float32,tf.float32,tf.float32])
gt_ab_313.set_shape(inputs['images'].get_shape())

In [8]:
gt_ab_313

<tf.Tensor 'PyFunc_8:1' shape=(256, 24, 24, 3) dtype=float32>

In [ ]:
with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    a = inputs['images'].eval()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [18]:
queue